In [1]:
item = {'sku_count': 750,
 'page_count': 8,
 'page': 1,
 'page_url': 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&parttype=Radiator%2520Fan%2520Assembly&start=0&num=100',
 'no': 1,
 'url': 'https://www.dormanproducts.com/p-76729-621-916.aspx?parttype=Radiator%2520Fan%2520Assembly&origin=keyword'}

item

{'sku_count': 750,
 'page_count': 8,
 'page': 1,
 'page_url': 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&parttype=Radiator%2520Fan%2520Assembly&start=0&num=100',
 'no': 1,
 'url': 'https://www.dormanproducts.com/p-76729-621-916.aspx?parttype=Radiator%2520Fan%2520Assembly&origin=keyword'}

In [2]:
import requests

import sys
sys.path.append('../00.Tools')
from configuration import generate_header, generate_proxy

In [3]:
response = requests.get(item['url'],
                        headers=generate_header(**{'Accept': 'text/html',
                                                   'Accept-Language': 'en',
                                                   'Cache-Control': 'no-cache',
                                                   'Pragma': 'no-cache',
                                                   'Sec-Fetch-Dest': 'document',
                                                   'Sec-Fetch-Mode': 'navigate',
                                                   'Sec-Fetch-Site': 'same-origin',
                                                   'Sec-Fetch-User': '?1',
                                                   'Upgrade-Insecure-Requests': '1'}),
                        proxies=generate_proxy(),
                        timeout=(10, 15),
                        allow_redirects=False)

response.status_code

200

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(response.text, 'html5lib')

with open('./3.part.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Radiator Fan Assembly | 621-916 | Radiator Fan Assembly Without Controller | Dorman Products
  </title>
  <meta charset="utf-8"/>
  <meta content="no-cache, no-store, must-revalidate" http-equiv="cache-control"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="-1" http-equiv="expires"/>
  <meta content="Dorman Products - 621-916 : Radiator Fan Assembly Without Controller. This engine cooling fan assembly is precision-engineered to match the fit and performance of the stock radiator fan on specific vehicle years, makes and models. It has been rigorously tested to ensure it cools as effectively as the original part." name="description"/>
  <meta content="621-916,  Engine, Cooling, Fan, Assembly, Fans, Assemblies, Rad, Radiator, Radiator Cooling Fan, Rad Fan, Rad Fan Assembly, AC Fan, Air Conditioning Fan, Condenser Fan Assembly" name="keywords"/>


In [6]:
from parsel import Selector

In [7]:
selector = Selector(text=str(soup))

selector

<Selector query=None data='<html lang="en"><head>\n    <title>Rad...'>

In [8]:
list_type = selector.xpath('//span[@id="lblPartTypeName"]/text()')

list_type

[<Selector query='//span[@id="lblPartTypeName"]/text()' data='Radiator Fan Assembly'>]

In [9]:
not list_type

False

In [10]:
type_ = list_type.get().strip()

type_

'Radiator Fan Assembly'

In [11]:
dict_src, i, list_href = {}, 0, list(reversed([href.strip() for href in selector.xpath('//a[@data-zoom-id="ProductPic"]/@href').getall()]))
if list_href:
    for href in list_href:
        if href.startswith('http'):
            dict_src[str(i)] = href
            i += 1
else:
    list_href = [href.strip() for href in selector.xpath('//a[@id="ProductPic"]/@href').getall()]
    if list_href:
        dict_src['0'] = list_href[0]

dict_src

{'0': 'https://static.dormanproducts.com/images/product/medium/621-916-001.JPG',
 '1': 'https://static.dormanproducts.com/images/product/medium/621-916-009.JPG'}

In [12]:
brand = selector.xpath('//span[@id="lblBrandName"]/text()').get().strip()

brand

'Dorman - OE Solutions'

In [13]:
part_number = selector.xpath('//span[@id="lblProductName"]/text()').get().strip()

part_number

'621-916'

In [14]:
description = selector.xpath('//span[@id="lblProductDesc"]/text()').get().strip()

description

'Radiator Fan Assembly Without Controller'

In [15]:
list_text = [text.strip() for text in selector.xpath('//div[@id="divAppSummary"]/p/text()').getall()]
while '' in list_text:
    list_text.remove('')

application_summary = ';'.join(list_text)

application_summary

'Chevrolet Chevy 2012-96, Chevrolet Chevy Pickup 2003-99'

In [16]:
from lxml import etree
import re
import html

In [17]:
list_div = selector.xpath('//div[@class="alert alert-warning"]')
list_text = [etree.tostring(div.root).decode('utf-8') for div in list_div]
list_text = [re.sub(r'<.*?>', '', text) for text in list_text]
list_text = [re.sub(r' +', ' ', text) for text in list_text]
list_text = [html.unescape(text).strip() for text in list_text]

warning = ''
for i, text in enumerate(list_text):
    warning += f'{i + 1}. {text}\n'

print(warning.strip())

In [18]:
from urllib.parse import urljoin
from urllib.parse import urlsplit
from urllib.parse import parse_qs
from urllib.parse import urlunsplit
from urllib.parse import urlencode

In [19]:
list_value = selector.xpath('//input[@id="hidfDetailApp"]/@value').getall()
if list_value:
    compatibility_home = urljoin(item['url'], list_value[0].strip())
    tuple_ = urlsplit(compatibility_home)
    dict_query = {key: value[0] for key, value in parse_qs(tuple_.query).items()}
    dict_query['start'] = 0
    dict_query['num'] = 50
    compatibility_home = urlunsplit((tuple_.scheme, tuple_.netloc, tuple_.path, urlencode(dict_query), tuple_.fragment))
else:
    compatibility_home = ''

compatibility_home

'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76729&PartType=Radiator+Fan+Assembly&start=0&num=50'

In [20]:
dict_key, list_tr = {}, selector.xpath('//section[@id="productSpec"]/div/table/tbody/tr')
for tr in list_tr:
    key = tr.xpath('./th/text()').get().strip().removesuffix(':').strip()

    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_specification, dict_count = {}, {}
for tr in list_tr:
    key, value = tr.xpath('./th/text()').get().strip().removesuffix(':').strip(), tr.xpath('./td/text()').get().strip()

    if dict_key[key] == 1:
        dict_specification[key] = 'nan' if value == '' else value
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_specification[f'{key}_{dict_count[key]}'] = 'nan' if value == '' else value

dict_specification

{'Blade Material': 'Plastic',
 'Connector Gender': 'Male',
 'Connector Quantity': '1',
 'Fan Quantity': '1',
 'Motor Type': 'Brushed',
 'Package Contents': '1 Radiator Fan Assembly',
 'Package Quantity': '1',
 'Packaging Type': 'Box',
 'Shroud Color': 'Black',
 'Shroud Included': 'Yes',
 'Shroud Material': 'Plastic',
 'Terminal Quantity': '2'}

In [21]:
list_text = selector.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()').getall()
list_text = [text.replace(' ', '') for text in list_text]
list_text = [text.replace('-', '') for text in list_text]
oem = ';'.join(list(dict.fromkeys(list_text)))

oem

'93803566;90410053;93230490'

In [22]:
import json

In [23]:
{'sku_count': item['sku_count'],
 'page_count': item['page_count'],
 'page': item['page'],
 'page_url': item['page_url'],
 'no': item['no'],
 'url': item['url'],
 'picture': '',
 'type_': type_,
 'json_src': json.dumps(dict_src),
 'brand': brand,
 'part_number': part_number,
 'description': description,
 'application_summary': application_summary,
 'warning': warning.strip(),
 'compatibility_home': compatibility_home,
 'json_specification': json.dumps(dict_specification),
 'oem': oem}

{'sku_count': 750,
 'page_count': 8,
 'page': 1,
 'page_url': 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&parttype=Radiator%2520Fan%2520Assembly&start=0&num=100',
 'no': 1,
 'url': 'https://www.dormanproducts.com/p-76729-621-916.aspx?parttype=Radiator%2520Fan%2520Assembly&origin=keyword',
 'picture': '',
 'type_': 'Radiator Fan Assembly',
 'json_src': '{"0": "https://static.dormanproducts.com/images/product/medium/621-916-001.JPG", "1": "https://static.dormanproducts.com/images/product/medium/621-916-009.JPG"}',
 'brand': 'Dorman - OE Solutions',
 'part_number': '621-916',
 'description': 'Radiator Fan Assembly Without Controller',
 'application_summary': 'Chevrolet Chevy 2012-96, Chevrolet Chevy Pickup 2003-99',
 'warning': '',
 'compatibility_home': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76729&PartType=Radiator+Fan+Assembly&start=0&num=50',
 'json_specification': '{"Blade Material": "Plastic", "Connector Gender": "Male", "Connector Q

In [24]:
compatibility_home == ''

False

In [25]:
{'sku_count': item['sku_count'],
 'page_count': item['page_count'],
 'page': item['page'],
 'page_url': item['page_url'],
 'no': item['no'],
 'url': item['url'],
 'picture': '',
 'type_': type_,
 'json_src': json.dumps(dict_src),
 'brand': brand,
 'part_number': part_number,
 'description': description,
 'application_summary': application_summary,
 'warning': warning.strip(),
 'compatibility_home': compatibility_home,
 'json_specification': json.dumps(dict_specification),
 'oem': oem,
 'compatibility_page': 1,
 'compatibility_url': compatibility_home}

{'sku_count': 750,
 'page_count': 8,
 'page': 1,
 'page_url': 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&parttype=Radiator%2520Fan%2520Assembly&start=0&num=100',
 'no': 1,
 'url': 'https://www.dormanproducts.com/p-76729-621-916.aspx?parttype=Radiator%2520Fan%2520Assembly&origin=keyword',
 'picture': '',
 'type_': 'Radiator Fan Assembly',
 'json_src': '{"0": "https://static.dormanproducts.com/images/product/medium/621-916-001.JPG", "1": "https://static.dormanproducts.com/images/product/medium/621-916-009.JPG"}',
 'brand': 'Dorman - OE Solutions',
 'part_number': '621-916',
 'description': 'Radiator Fan Assembly Without Controller',
 'application_summary': 'Chevrolet Chevy 2012-96, Chevrolet Chevy Pickup 2003-99',
 'warning': '',
 'compatibility_home': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76729&PartType=Radiator+Fan+Assembly&start=0&num=50',
 'json_specification': '{"Blade Material": "Plastic", "Connector Gender": "Male", "Connector Q